# 2D Bridge Frame Analysis


In [ ]:
#@title Update and use python 3.12 for performance boosts
%%capture

# !pip install symengine
!pip install cython
!pip install dill
!pip install huggingface_hub
# !pip install numpy --upgrade


# Install python 3.12 (for faster performance)
!sudo apt-get update -y
!sudo apt-get install python3.12
!sudo apt-get install python3-pip

# Change default python3 to 3.12
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.12 1

# Confirm version
!python3 --version
# Python 3.12.1

In [ ]:
#@title Download the SMH dataset and move it to the current working directory
%%capture

# For huggingface
from google.colab import userdata

import os
import shutil
from huggingface_hub import snapshot_download

repo_id = "your_repo_id"  # Replace with the actual repository ID
repo_type = "dataset"  # Specify the repository type (model or dataset)
destination_path = "SMH"
# token = "your_hugging_face_token"  # Replace with your actual Hugging Face token
token = userdata.get('HF_TOKEN')     # If secrets use this

if not os.path.exists(destination_path):
    os.makedirs(destination_path)

# Download the entire dataset to the specified destination path
snapshot_download(repo_id, repo_type=repo_type, revision="main", token=token, local_dir=destination_path)

# Move the 'combos' folder to the current working directory
combos_folder_path = os.path.join(destination_path, "combos")
if os.path.exists(combos_folder_path):
    shutil.move(combos_folder_path, os.getcwd())
    print(f"Moved 'combos' folder to the current working directory.")
else:
    print("The 'combos' folder does not exist in the downloaded dataset.")


def move_contents_out_and_delete_folder(folderpath):
    # Ensure the folderpath exists and is a directory
    if not os.path.exists(folderpath):
        print(f"The specified path does not exist: {folderpath}")
        return

    if not os.path.isdir(folderpath):
        print(f"The specified path is not a directory: {folderpath}")
        return

    # Get the parent directory of the folderpath
    parent_dir = os.path.dirname(folderpath)

    # Move all contents (files and subdirectories) from the folderpath to the parent directory
    for item in os.listdir(folderpath):
        src_path = os.path.join(folderpath, item)
        dst_path = os.path.join(parent_dir, item)

        # Move the file or directory
        shutil.move(src_path, dst_path)

    # Remove the now empty folder
    os.rmdir(folderpath)
    print(f"All contents moved and folder '{folderpath}' deleted.")

move_contents_out_and_delete_folder(f"./{destination_path}")

In [ ]:
#@title Importing Libraries
# magic to reload modules when they change
%load_ext autoreload
%autoreload 2

import os
import re
import math
import time
import importlib
import numpy as np
import sympy as sp
from google.colab import files
import matplotlib.pyplot as plt
from sympy import Matrix, lambdify
from sympy.utilities.codegen import codegen
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor


import logging
from utils.geometric import *
from truss_constraints import *
from utils.truss_helpers import *

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')



# Building the truss structure


In [ ]:
#@title Choose which file to analyze
csv_name = "combos/combo4_60.csv"
X_disp_tot = "X_combo_output"
E_combo_output = "E_combo_output"
A_combo_output = "A_combo_output"
J_combo_output = "J_combo_output"
h_combo_output = "h_combo_output"
T_combo_output = "T_combo_output"
w_combo_output = "w_combo_output"

plot_emphasis = 1

# Extract the first number and everything from that number till the end (excluding extension)
match = re.search(r'\d.*?(?=\.)', csv_name)

if match:
    extracted_sequence = match.group()
    X_disp_tot += extracted_sequence
    E_combo_output += extracted_sequence
    A_combo_output += extracted_sequence
    J_combo_output += extracted_sequence
    h_combo_output += extracted_sequence
    T_combo_output += extracted_sequence
    w_combo_output += extracted_sequence

X_output_file = X_disp_tot + ".csv"
E_output_file = E_combo_output + ".csv"
A_output_file = A_combo_output + ".csv"
J_output_file = J_combo_output + ".csv"
h_output_file = h_combo_output + ".csv"
T_output_file = T_combo_output + ".csv"
w_output_file = w_combo_output + ".csv"
X_output_file

('X_combo_output4_60.csv',
 'E_combo_output4_60.csv',
 'A_combo_output4_60.csv',
 'J_combo_output4_60.csv',
 'h_combo_output4_60.csv',
 'T_combo_output4_60.csv',
 'w_combo_output4_60.csv')

# Bridge Structure

In [ ]:
#@title Choose which file to analyze
csv_name = "combos/bridge_combo4567.csv"
Y_disp_tot = "Y_combo_output"
X_disp_tot = "X_combo_output"
E_combo_output = "E_combo_output"
A_combo_output = "A_combo_output"
J_combo_output = "J_combo_output"
h_combo_output = "h_combo_output"
T_combo_output = "T_combo_output"
w_combo_output = "w_combo_output"

plot_emphasis = 1

# Extract the first number and everything from that number till the end (excluding extension)
match = re.search(r'\d.*?(?=\.)', csv_name)

if match:
    extracted_sequence = match.group()
    Y_disp_tot += extracted_sequence
    X_disp_tot += extracted_sequence
    E_combo_output += extracted_sequence
    A_combo_output += extracted_sequence
    J_combo_output += extracted_sequence
    h_combo_output += extracted_sequence
    T_combo_output += extracted_sequence
    w_combo_output += extracted_sequence

Y_output_file = Y_disp_tot + ".csv"
X_output_file = X_disp_tot + ".csv"
E_output_file = E_combo_output + ".csv"
A_output_file = A_combo_output + ".csv"
J_output_file = J_combo_output + ".csv"
h_output_file = h_combo_output + ".csv"
T_output_file = T_combo_output + ".csv"
w_output_file = w_combo_output + ".csv"
X_output_file

'X_combo_output4567.csv'

## New method with Cython

In [ ]:
# Bernoulli or timoshenko, newton or lagrangian
beam_type = "bernoulli"
use_lagrangian = False

# Basic geometric functions already defined previously
span = 25
angle = 45
beam_length = 2.5
skip_rod = []
truss_mode = "warren" # warren, pratt, howe

# Calculation of bridge geometry
height, spacing, diag = calculate_bridge(span, angle, beam_length, truss_mode)
n_columns, n_nod_tot, n_rods, n_beams, n_ele_tot, n_bot_beams = calculate_essential_elements(span, spacing, truss_mode, skip_rod)

# Recalculates the skip_rod to match the truss design
skip_rod = truss_design(n_bot_beams, n_rods, truss_mode)
n_columns, n_nod_tot, n_rods, n_beams, n_ele_tot, n_bot_beams = calculate_essential_elements(span, spacing, truss_mode, skip_rod)


# Calculation of nodal coordinates and construction of par, pel matrices and element node indices
# Building priorty: Beams -> Columns -> Rods
nodal_coord, par, pel, ele_nod, n_par_tot = calculate_element_node(span, spacing, height, n_dim, n_par_nod, truss_mode, skip_rod)

# Seperating nodal coordinates into x and y coordinates
X = np.zeros(n_nod_tot, dtype=float)
Y = np.zeros(n_nod_tot, dtype=float)
X[:] = nodal_coord[:, 0]
Y[:] = nodal_coord[:, 1]


# -------- BOUNDARY CONDITIONS --------
# Focuses only on the first and last node connect to the ground (roller, pin, fixed)
W = boundary_conditions(n_bot_beams, n_par_nod, n_nod_tot, supports=["pin", "roller"])
pel = pel.astype(np.int32)
W = W.astype(np.int32)
ele_nod = ele_nod.astype(np.int32)

# # -------- ELEMENT CHARACTERIZING DATA --------
h = np.zeros(n_ele_tot, dtype=np.float32)
J = np.zeros(n_ele_tot, dtype=np.float32)
A = np.zeros(n_ele_tot, dtype=np.float32)
beta = np.zeros(n_ele_tot,dtype=np.float32)
ro = np.zeros(n_ele_tot,dtype=np.float32)
E = np.zeros(n_ele_tot, dtype=np.float32)


# # -------- ELEMENT PROPERTIES --------
J, A, h, beta, ro, E, G = calculate_element_properties(n_columns, n_beams, diag, spacing, height, J, A, h, beta, ro, E, X, Y, ele_nod, shear_mod,
                                                 width_properties, height_properties, unit_weight_properties, elastic_mod_properties)


# -------- SYMBOLIC CALCULATIONS (Moved outside loop) --------
# Define the symbols
(x, xi, h_e, beta_e, beta_curr, qe, a_arr, b_arr,
    c_arr, d_arr, e_arr, A_e, E_e, J_e, ro_e, T, fo_E, X_old, Y_old,
        Qglo_pel_curr, w_arr, r_arr, f_arr, g_arr) = initialize_symbols(n_par_ele)

# Compute the local displacement expressions
v1, u1, v2, u2 = compute_v_u(qe, beta_e)

# Calculate the beam displacement expressions
(ve_beam_func, ue_beam_func, ve_beam, ue_beam, ve_rod_func,
    ue_rod_func, ve_rod, ue_rod, alpha_e_beam) = calculate_displacement_equations(x, xi, h_e, beta_e, qe, a_arr, b_arr, c_arr, d_arr, e_arr,
                                     beam_type, use_lagrangian)

# Calculate the energies of beams and rods
pot_beam, kin_beam, pot_rod, kin_rod = calculate_energies(beam_type, ve_beam, ue_beam, alpha_e_beam, ve_rod,
                                                          ue_rod, x, h_e, E_e, J_e, A_e, ro_e, G, k_shear)



# -------- STIFFNESS AND MASS SYMBOLIC MATRICES (MOVED OUT OF LOOP) --------
# Calculate the lambdified expressions of stiffness and mass matrices
(K_beam, M_beam, K_rod, M_rod, K_beam_func, M_beam_func,
 K_rod_func, M_rod_func) = construct_lambdified_matrices(n_par_ele, pot_beam, kin_beam, pot_rod, kin_rod,
                                                         qe, h_e, A_e, E_e, J_e, beta_e, ro_e)

# -------- GLOBAL DISPLACEMENTS (MOVED OUT OF LOOP) --------
# Defines local displacements and calculates global displacements symbolic lambda functions
(X_new_beam, Y_new_beam, X_new_rod, Y_new_rod, X_new_beam_func, Y_new_beam_func,
 X_new_rod_func, Y_new_rod_func) = calculate_global_displacements(Qglo_pel_curr, beta_e, h_e, x, xi, f_arr,
                                                                         g_arr, w_arr, r_arr, beam_type,
                                                                         X_old, Y_old, use_lagrangian)


#Turn the symbolic functions into lambdified functions suitable for Cython
functions = [
    ("K_beam_func", K_beam),
    ("M_beam_func", M_beam),
    ("K_rod_func", K_rod),
    ("M_rod_func", M_rod),
]

# Generate C code
codegen(
    functions,
    language="C",
    project="utils/cython",
    to_files=True
)


functions = [
    ("X_beam_func", X_new_beam),
    ("Y_beam_func", Y_new_beam),
    ("X_rod_func", X_new_rod),
    ("Y_rod_func", Y_new_rod),
]

# Generate C code
codegen(
    functions,
    language="C",
    project="utils/cython",
    to_files=True
)


def replace_include_guards(file_path):
    with open(file_path, 'r') as file:
        content = file.read()

    # Replace slashes with underscores in the include guards
    content = content.replace('UTILS/CYTHON__', 'UTILS_CYTHON__')

    # Add whitespace after the define directive if missing
    content = content.replace('#defineUTILS_CYTHON__', '#define UTILS_CYTHON__')

    with open(file_path, 'w') as file:
        file.write(content)

# Paths to the header files
header_files = [
    "K_beam_func.h",
    "X_beam_func.h"
]

# Move the generated C files and modify the include guards in the header files
for header_file in header_files:
    source_path = header_file
    dest_path = os.path.join("utils/cython", header_file)

    # Replace include guards
    replace_include_guards(source_path)

    # Move the header file to the destination
    shutil.move(source_path, dest_path)

# Move the generated C files
shutil.move("K_beam_func.c", "utils/cython/K_beam_func.c")
shutil.move("X_beam_func.c", "utils/cython/X_beam_func.c")

!rm -r utils/cython/build
!cd utils/cython && python setup.py build_ext --inplace

from utils.cython.assemble_matrices import assemble_global_matrices_cython, apply_boundary_conditions_cython, compute_element_matrices_cython, compute_element_coordinates_cython

In [ ]:
#@title Building a truss structure for a bridge (Cython)

#@title Main function to run
start_time = time.time()
# Read the combo file
modulus_combo = np.loadtxt(csv_name, delimiter=',', skiprows=1)
print(len(modulus_combo))

# Beam/col indices and rod_indices
beam_indices = np.arange(n_ele_tot - n_rods, dtype=np.int32)
rod_indices = np.arange(n_ele_tot - n_rods, n_ele_tot, dtype=np.int32)

# -------- FINAL RESULT ARRAYS --------
# Store the total displacements
Y_disp_tot = np.zeros((len(modulus_combo), n_plots, n_ele_tot))
X_disp_tot = np.zeros((len(modulus_combo), n_plots, n_ele_tot))
E_output = np.zeros((len(modulus_combo), n_ele_tot))
A_output = np.zeros((len(modulus_combo), n_ele_tot))
J_output = np.zeros((len(modulus_combo), n_ele_tot))
h_output = np.zeros((len(modulus_combo), n_ele_tot))
T_output = np.zeros((len(modulus_combo), n_plots, n_ele_tot))
w_output = np.zeros((len(modulus_combo), n_plots, n_ele_tot))

amplifier = int(span/spacing)
for l in range(1):
    for iterr, E_combos in enumerate(modulus_combo):
        E[:] = E_combos[:]

        # -------- ASSEMBLE GLOBAL STIFFNESS MATRIX --------
        element_matrices = compute_element_matrices_cython(
            n_ele_tot, beam_indices, rod_indices,
            h, A, E, J, beta, ro
        )
        K, M = assemble_global_matrices_cython(element_matrices, pel, n_par_tot)
        K, M = apply_boundary_conditions_cython(K, M, W, tol=1e-5)


        # -------- EIGENVALUE PROBLEM --------
        # Compute eigenvalues (lamb) and eigenvectors (phis)
        # Already taken the real part and normalized
        # Compute eigenvalues and eigenvectors using scipy method without filtering
        lamb_r, phis_norm = compute_eigenvalues_and_eigenvectors(K, M, method='numpy', filter_numerical_stability=True, threshold=1e-10)
        index_modes, sorted_period, period = get_mode_indices(lamb_r, phis_norm, n_plots)

        # Extract lambdas and corresponding eigenvectors
        lamb_plots = lamb_r[index_modes]
        phis_plots = phis_norm[:, index_modes]

        # Define lists
        X_new_sub, Y_new_sub, X_disp, Y_disp = compute_element_coordinates_cython(
                    n_plots, n_ele_tot, n_rods, n_discritizations, span, spacing,
                    X, Y, ele_nod, beta, h, pel, phis_plots, use_lagrangian)


        Y_disp_tot[iterr] = Y_disp
        X_disp_tot[iterr] = X_disp

        E_output[iterr] = E
        A_output[iterr] = A
        J_output[iterr] = J
        h_output[iterr] = h
        for i in range(n_plots):
          T_output[iterr][i] = np.full(n_ele_tot, period[index_modes[i]])
          w_output[iterr][i] = np.full(n_ele_tot, math.sqrt(lamb_plots[i]))

        # print(X_disp[0,:])
end_time = time.time()
print("Elapsed time:", end_time - start_time, "seconds")
# 190k took 4 hours (original) & 118711 in 28 mins (new) => 46 mins for 190k (new)
np.savetxt(Y_output_file, Y_disp_tot.reshape(len(modulus_combo), -1), delimiter=",")
np.savetxt(X_output_file, X_disp_tot.reshape(len(modulus_combo), -1), delimiter=",")
np.savetxt(E_output_file, E_output, delimiter=",")
np.savetxt(A_output_file, A_output, delimiter=",")
np.savetxt(J_output_file, J_output, delimiter=",")
np.savetxt(h_output_file, h_output, delimiter=",")
np.savetxt(T_output_file, T_output.reshape(len(modulus_combo), -1), delimiter=",")
np.savetxt(w_output_file, w_output.reshape(len(modulus_combo), -1), delimiter=",")

118711
Elapsed time: 1715.0391359329224 seconds


In [ ]:
files.download(Y_output_file)
files.download(X_output_file)
files.download(E_output_file)
files.download(A_output_file)
files.download(J_output_file)
files.download(h_output_file)
files.download(T_output_file)
files.download(w_output_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>